In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import salem

In [4]:
ds = xr.open_dataset(r'C:\Users\USER\JupyterLab\Climaturk_Site\Docs\STORM_DATA\precipitation_2010_2021_data.nc')

In [5]:
#merging part is not necessary if you dont take the year 2021: ECMWF's Studpic expver variable...
#tp = xr.merge([ds.sel(expver=1), ds.sel(expver=5)], compat='override')['tp'].sel(time = ds['tp'].time.dt.year<=2020)
tp = ds['tp'].sel(time = ds['tp'].time.dt.year<=2020)


In [6]:
shapefile = salem.read_shapefile(r'C:\Users\USER\Documents\BitirmeProjesiKutay\Shapefile_Turkey\turkey_administrativelevels0_1_2\tur_polbnda_adm1.shp')
tp = tp.salem.roi(shape=shapefile)

In [7]:
tp = tp.resample(time='1D').sum()

In [8]:
def find_top_ten(data, year, month):
    
    ds = data.sel(time=data.time.dt.year==year) 
    #get the requested year
    ds = ds.sel(time=ds.time.dt.month==month)
    #get the requested month
    
    half_turkey_latitude_index = int(len(ds.latitude)/2)
    half_turkey_longitude_index = int(len(ds.longitude)/2)
    
    left_upper_turkey  = ds.isel(latitude=slice(0, half_turkey_latitude_index), longitude=slice(0, half_turkey_longitude_index)).sum(dim=['latitude', 'longitude'])
    left_bottom_turkey = ds.isel(latitude=slice(half_turkey_latitude_index, -1), longitude=slice(0, half_turkey_longitude_index)).sum(dim=['latitude', 'longitude'])
    right_upper_turkey = ds.isel(latitude=slice(0, half_turkey_latitude_index), longitude=slice(half_turkey_longitude_index, -1)).sum(dim=['latitude', 'longitude'])
    right_bottom_turkey = ds.isel(latitude=slice(half_turkey_latitude_index, -1), longitude=slice(half_turkey_longitude_index, -1)).sum(dim=['latitude', 'longitude'])
    total_turkey = ds.sum(dim=['latitude', 'longitude'])
    #get data for different sections sum daily values over the latitudes and longitudes

    section_list = [left_upper_turkey, left_bottom_turkey, right_upper_turkey, 
                    right_bottom_turkey, total_turkey]
    section_names = ['left_upper_turkey', 'left_bottom_turkey', 'right_upper_turkey', 
                     'right_bottom_turkey', 'total_turkey']
    
    sorted_pandas_values = {}
    #dictionary to store sorted values for given year and day
    sorted_pandas_times = {}
    #dictionary to store sorted dates for given year and day

    for i,j in zip(section_list, section_names):
        sorted_data = i.sortby(i, ascending=False)[0:10]#top 10
        sorted_time = sorted_data.time
        sorted_pandas_times['{}'.format(j)] = sorted_time
        sorted_pandas_values['{}'.format(j)] = sorted_data
    sorted_pandas_times = pd.DataFrame.from_dict(sorted_pandas_times)
    sorted_pandas_values = pd.DataFrame.from_dict(sorted_pandas_values)

    return sorted_pandas_times, sorted_pandas_values

In [9]:
#the dates
dates = []
dates_s = []
for y in np.arange(2010, 2020):
    for m in np.arange(1, 13):
        
        month_map_str = {'01':'January',
                         '02':'February',
                         '03':'March',
                         '04':'April',
                         '05':'May',
                         '06':'June',
                         '07':'July',
                         '08':'August',
                         '09':'September',
                         '10':'October',
                         '11':'November',
                         '12':'December'}
        
        # ay ve yıl bir tane 
        month = [str(i)[5:7] for i in find_top_ten(tp, y, m)[0].iloc[0,:-1].values][0]
        
        # günler farkedecek, günler aynı ise onlardan birisini alma
        days = [str(i)[8:10] for i in find_top_ten(tp, y, m)[0].iloc[0,:-1].values]
        
        lut_date = [str(y)+month+days[0], str(y)+month_map_str[month]+days[0]]
        lbt_date = [str(y)+month+days[1], str(y)+month_map_str[month]+days[1]]
        rup_date = [str(y)+month+days[2], str(y)+month_map_str[month]+days[2]]
        rbt_date = [str(y)+month+days[3], str(y)+month_map_str[month]+days[3]]

        dates.append(lut_date[0])
        dates.append(lbt_date[0])
        dates.append(rup_date[0])
        dates.append(rbt_date[0])
        
        dates_s.append(lut_date[1])
        dates_s.append(lbt_date[1])
        dates_s.append(rup_date[1])
        dates_s.append(rbt_date[1])

        
    

dates = list(set(dates))
dates_s = list(set(dates_s))

In [18]:
t = {'year': {
        "1": ['1', '5', '6'],
        '2': ['3', '4', '5'],
            '2': ['3', '4', '5'],
            '3': ['3', '4', '5'],
            '4': ['3', '4', '5'],
            '5': ['3', '4', '5'],
            '6': ['3', '4', '5'],
            '7': ['3', '4', '5'],
            '8': ['3', '4', '5'],

        '12': [],
}
}

In [36]:
pd_data

,dates,dates_s,download_done,plot_done
0,20181025,2018April10,0,0
1,20100805,2010July16,0,0
2,20130917,2016November28,0,0
3,20151007,2016May27,0,0
4,20130831,2014August05,0,0
...,...,...,...,...
368,20170728,2014April28,0,0
369,20110219,2019July11,0,0
370,20140509,2017August04,0,0
371,20120411,2011September24,0,0


In [10]:
pd_dict = {'dates':dates,
           'dates_s':dates_s,
           'download_done': np.zeros((len(dates)), dtype = int),
           'plot_done': np.zeros((len(dates)), dtype=int)}

pd_data = pd.DataFrame.from_dict(pd_dict)

In [37]:
outer_dict = {}

for y in range(2010, 2020):
    
    inner_dict = {}
    for m in range(1, 13):
        
        inner_dict[str(m)] = []
        for d in range(1, 31):
            
            for i in range(len(pd_data)):
                
                year = str(pd_data['dates'][i][:4])
                month = int(str(pd_data['dates'][i][4:6]))
                day = int(str(pd_data['dates'][i][6:]))

                
                if str(year) + str(month) + str(day) == str(y) + str(m) + str(d):
                    
                    inner_dict[str(m)].append(str(d))

    outer_dict[str(y)] = inner_dict

In [38]:
outer_dict

{'2010': {'1': ['20', '22', '24'],
  '2': ['2', '3', '8', '22'],
  '3': ['3', '5', '6', '15'],
  '4': ['11', '12', '20', '21'],
  '5': ['1', '22', '23'],
  '6': ['6', '10', '24'],
  '7': ['8', '16', '17', '24'],
  '8': ['5', '26'],
  '9': ['13', '30'],
  '10': ['14', '28', '29'],
  '11': ['25'],
  '12': ['2', '4', '10', '11']},
 '2011': {'1': ['13', '25', '27', '30'],
  '2': ['19', '20', '26'],
  '3': ['7', '8', '9'],
  '4': ['2', '10', '22'],
  '5': ['4', '5', '12'],
  '6': ['10', '11', '13'],
  '7': ['1', '2', '16', '19'],
  '8': ['12'],
  '9': ['22', '24', '30'],
  '10': ['10', '13', '19'],
  '11': ['3', '14'],
  '12': ['5', '7', '23', '24']},
 '2012': {'1': ['7', '16', '18', '24', '27'],
  '2': ['7', '17', '28'],
  '3': ['3', '12', '13', '14'],
  '4': ['10', '11', '14', '19'],
  '5': ['3', '18', '25', '28'],
  '6': ['4', '6', '28'],
  '7': ['3', '8', '30'],
  '8': ['1', '2', '6', '28'],
  '9': ['2', '19', '21', '22'],
  '10': ['11', '21', '25'],
  '11': ['6', '8', '11'],
  '12': ['

todays_date = '10042021'
np.save( r'C:\Users\USER\JupyterLab\Climaturk_Site\Codes\NewEngineeering_Codes\DateList_Synoptic_{}'.format(todays_date), pd_data.to_numpy())
np.save( r'C:\Users\USER\JupyterLab\Climaturk_Site\Codes\NewEngineeering_Codes\DateList_Thermodynamic_{}'.format(todays_date), pd_data.to_numpy())